In [1]:
import imperial_materials_simulation as ims
import numpy as np
import time

# Legacy Tests

This code makes sure the current physics module gives the same results as the course's original implementations.

In [2]:
n_atoms = 25
equilibrium_bond_length = 1.53
spring_constant = 15.18
sigma = 4.5
epsilon = 0.00485678
positions = np.zeros(shape=(n_atoms, 3))
positions[:, 1] = np.linspace(0, (equilibrium_bond_length+np.random.rand()/10)-1, num=n_atoms)

### Bonding Forces

In [3]:
def GetBondingEnergy(pos, calculate_force=True): #old function
#
    potential_bond = 0.0
#
    bond = np.zeros(3)
    bonddir = np.zeros(3)
    force_bond = np.zeros((n_atoms,3))
#
    for i in range(n_atoms-1):
        bond = pos[i+1]-pos[i]
        bondlength = np.linalg.norm(bond)
        bonddir = bond/bondlength
        dbondlength = bondlength - equilibrium_bond_length
        potential_bond = potential_bond + spring_constant*(dbondlength**2.0)/2.0
        if calculate_force:
            force_bond[i] = force_bond[i] + spring_constant*dbondlength*bonddir
            force_bond[i+1] = force_bond[i+1] - spring_constant*dbondlength*bonddir
    
    return potential_bond, force_bond

In [4]:
new_forces, new_potential = ims.physics.get_bonding_interactions(positions, equilibrium_bond_length, spring_constant)
old_potential, old_forces = GetBondingEnergy(positions)
assert np.allclose(new_potential, old_potential)
assert np.allclose(new_forces, old_forces)

In [5]:
start = time.perf_counter()
for i in range(1_000):
    GetBondingEnergy(positions)
old_run_time_b = time.perf_counter() - start
old_run_time_b

0.17364789987914264

In [6]:
start = time.perf_counter()
for i in range(1_000):
    ims.physics.get_bonding_interactions(positions, equilibrium_bond_length, spring_constant)
new_run_time_b = time.perf_counter() - start
new_run_time_b

0.0055766000878065825

In [7]:
old_run_time_b / new_run_time_b

31.13866821091034

### Non-Bonding Forces

In [8]:
def GetNonBondingEnergy(pos, calculate_force=True): #depricated old function
#
    disp = np.zeros(3)
    dforce = np.zeros(3)
    force_nonbond = np.zeros((n_atoms,3))
#
    potential_nonbond = 0.0
    factor1 = -12.0*epsilon/(sigma*sigma)
    for i in range(n_atoms-1):
        for j in range(i+3,n_atoms):
            disp = pos[j]-pos[i]
            dist = np.linalg.norm(disp)
            squared = (sigma/dist)**2.0
            sixpower = squared**3.0 
            twelvepower = sixpower*sixpower
            potential_nonbond = potential_nonbond + epsilon*(twelvepower - 2.0*sixpower)
            if calculate_force:
                dforce  = factor1*squared*(twelvepower - sixpower)*disp
                force_nonbond[i] = force_nonbond[i] + dforce
                force_nonbond[j] = force_nonbond[j] - dforce
    return potential_nonbond, force_nonbond

In [9]:
def GetNonBondingEnergy_Carlos(pos, calculate_force=None): #old function
   #
      force_nonbond = np.zeros((n_atoms,3))
   #
      potential_nonbond = 0.0
      factor1 = -12.0*epsilon/(sigma*sigma)
      for i in range(n_atoms-1):
         
         disp_mat=pos[i+3:]-pos[i]
         #dist_sq=calc_dist(pos[i+3:],pos[i].reshape((1,3))).flatten()
         #potentials
         dist_sq=np.sum(disp_mat*disp_mat, axis=1)
         squared_mat = np.divide(sigma**2,dist_sq)
         sixpower_mat = squared_mat**3.0 
         twelvepower_mat = sixpower_mat*sixpower_mat
         potentials_nonbond_mat = epsilon*(twelvepower_mat - 2.0*sixpower_mat)
         potential_nonbond = potential_nonbond+potentials_nonbond_mat.sum()
         
         
         #
         dforce=disp_mat*(factor1*squared_mat*(twelvepower_mat - sixpower_mat))[:,None]
         force_nonbond[i]=force_nonbond[i]+np.sum(dforce,axis=0)
         force_nonbond[i+3:]=force_nonbond[i+3:]- dforce

      return potential_nonbond, force_nonbond

In [10]:
new_forces, new_potential = ims.physics.get_non_bonding_interactions(positions, epsilon, sigma)
old_potential, his_forces = GetNonBondingEnergy_Carlos(positions)
assert np.allclose(new_potential, old_potential)
assert np.allclose(new_forces, his_forces)

In [11]:
start = time.perf_counter()
for i in range(1_000):
    GetNonBondingEnergy_Carlos(positions)
old_run_time_nb = time.perf_counter() - start
old_run_time_nb

0.4762492999434471

In [12]:
start = time.perf_counter()
for i in range(1_000):
    ims.physics.get_non_bonding_interactions(positions, epsilon, sigma)
new_run_time_nb = time.perf_counter() - start
new_run_time_nb

0.04601630009710789

In [13]:
old_run_time_nb / new_run_time_nb

10.349578278532203

### Metropolis Monte Carlo Force Tracker

This just compares how much faster it is to only recalculate changed forces as compared to recalculating all forces.

In [14]:
class OldPotentialEnergyTracker():
    '''
    Utility class for efficiently tracking how total potential energy (bonding + Lennard Jones non-bonding) changes
    when a single atom is displaced.
    '''

    def __init__(self, positions: np.ndarray, epsilon: float, sigma: float, equilibrium_bond_length: float,
                spring_constant: float) -> None:
        'calculates key distances, lengths and total potential energy'
        self.positions = positions
        self.epsilon = epsilon
        self.sigma = sigma
        self.equilibrium_bond_length = equilibrium_bond_length
        self.spring_constant = spring_constant
        self.n_atoms = len(positions)

        # (n_atoms, n_atoms, 3) array where array[i,j] is the displacement between ith atom and jth atom
        self.displacements = positions.reshape((self.n_atoms, 1, 3)) - positions.reshape((1, self.n_atoms, 3))
        
        self.col_indexes, self.row_indexes = np.meshgrid(np.arange(self.n_atoms), np.arange(self.n_atoms))
        self.bonding_mask = (self.col_indexes-self.row_indexes) == 1 #all bonding interactions
        #half of non-bonding interactions (all that is needed to calculate non bonding potential energy)
        self.non_bonding_mask = (self.col_indexes-self.row_indexes) > 2 

        #should ignore distances between atom and itself (i==j). Set to 1 to prevent 0 division errors
        self.displacements[self.col_indexes==self.row_indexes] = 1
        self.lengths = np.linalg.norm(self.displacements, axis=2)

        bonding_extensions = self.lengths[self.bonding_mask] - equilibrium_bond_length
        bonding_potential =  np.sum(spring_constant/2 * bonding_extensions**2)
        six_power = (sigma/self.lengths[self.non_bonding_mask]) ** 6
        non_bonding_potential = np.sum(epsilon * (six_power**2 - 2*six_power))
        self.total_potential_energy = bonding_potential + non_bonding_potential
        
        #used in test_displacement method. allows for each pair to only be checked once. length between ij
        #calculated, but not length between ji. using the second length when only calculating potentials is redundant
        self.relevant_interactions = self.col_indexes > self.row_indexes

    def get_total_potential_energy(self):
        '''return total potential energy of molecule'''
        return self.total_potential_energy

    def test_displacement(self, atom_index: int, displacement: np.ndarray) -> float:
        '''stores temporary new positions, displacements, and lengths. Returns change in potential energy'''
        self.new_positions = self.positions.copy()
        self.new_positions[atom_index] += displacement

        self.new_displacements = self.displacements.copy()
        self.new_displacements[:, atom_index] = self.new_positions - self.new_positions[atom_index].reshape(1, 3)
        self.new_displacements[atom_index, :] = self.new_positions[atom_index].reshape(1, 3) - self.new_positions

        affected_interactions = (self.col_indexes == atom_index) | (self.row_indexes == atom_index)
        update_mask = self.relevant_interactions & affected_interactions
        self.new_lengths = self.lengths.copy()
        #axis=1 as slicing 3D array with boolean mask returns 2D array
        self.new_lengths[update_mask] = np.linalg.norm(self.new_displacements[update_mask], axis=1)

        bonding_extensions = self.new_lengths[self.bonding_mask] - self.equilibrium_bond_length
        bonding_potential =  np.sum(self.spring_constant/2 * bonding_extensions**2)
        six_power = (self.sigma/self.new_lengths[self.non_bonding_mask])**6
        non_bonding_potential = np.sum(self.epsilon * (six_power**2 - 2*six_power))
        self.new_total_potential_energy = bonding_potential + non_bonding_potential
        return self.new_total_potential_energy - self.total_potential_energy
    
    def accept_last_displacement(self) -> None:
        '''replaces internal positions, displacements, and lengths with values from last test displacement'''
        self.positions = self.new_positions
        self.displacements = self.new_displacements
        self.lengths = self.new_lengths
        self.total_potential_energy = self.new_total_potential_energy

In [15]:
F_b, PE_b = ims.physics.get_bonding_interactions(positions, equilibrium_bond_length, spring_constant)
F_nb, PE_nb = ims.physics.get_non_bonding_interactions(positions, epsilon, sigma)
true_PE = PE_b + PE_nb

energy_tracker = ims.physics.PotentialEnergyTracker(positions, epsilon, sigma, equilibrium_bond_length, spring_constant)
new_PE = energy_tracker.get_total_potential_energy()

assert np.allclose(true_PE, new_PE)

In [16]:
atom_index, displacement = 1, np.array([0.1, 1, -0.3])
new_positions = positions.copy()
new_positions[atom_index] += displacement

F_b, PE_b = ims.physics.get_bonding_interactions(new_positions, equilibrium_bond_length, spring_constant)
F_nb, PE_nb = ims.physics.get_non_bonding_interactions(new_positions, epsilon, sigma)
true_displaced_PE = PE_b + PE_nb
true_PE_change = true_displaced_PE - true_PE

new_PE_change = energy_tracker.test_displacement(atom_index, displacement)

assert np.allclose(true_PE_change, new_PE_change)

In [17]:
energy_tracker = OldPotentialEnergyTracker(positions, epsilon, sigma, equilibrium_bond_length, spring_constant)
start = time.perf_counter()
for i in range(5_000):
    energy_tracker.test_displacement(atom_index, displacement)
old_MMC_runtime = time.perf_counter() - start
old_MMC_runtime

0.25818620016798377

In [18]:
energy_tracker = ims.physics.PotentialEnergyTracker(positions, epsilon, sigma, equilibrium_bond_length, spring_constant)
start = time.perf_counter()
for i in range(5_000):
    energy_tracker.test_displacement(atom_index, displacement)
new_MMC_runtime = time.perf_counter() - start
new_MMC_runtime

0.02687479997985065

In [19]:
old_MMC_runtime / new_MMC_runtime

9.606999879498957